In [1]:
#all the needed Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
import re
import gensim
from gensim.parsing.preprocessing import remove_stopwords

In [2]:
df = pd.read_excel("ServiceNowQuestions_clean.xlsx")
df.drop('Tags', inplace=True, axis=1)
df.drop('index', inplace=True, axis=1)
df.drop('Question', inplace=True, axis=1)
df.drop('level_0', inplace=True, axis=1)

In [3]:
df

,Title,Answer
0,how to cut / trim the value short_description ...,So you need to write this in server-side scrip...
1,Query to find max date,"Hi Wade,I was going to try to step through tro..."
2,addQuery and addEncodedQuery queries not worki...,Please see below for how to troubleshoot and s...
3,Create object from XML attachment larger than 5MB,I have come across 2 solutions to address this...
4,buttons on email notification,"Hi User,Please follow below link or stepdhttps..."
...,...,...
5739,Replicate Hi Support Top Blue Box Announcement,"Hi David [davidsizu ],This is a custom impleme..."
5740,UI Action button visible to only assigned assi...,Try something like: gs.getUser().isMemeberOf(c...
5741,How to choose field type as email?As I can see...,This is a system limitation. Field validations...
5742,Service Portal Widget,"Its a bit much code to include here, so I have..."


In [4]:
def clean_sentence(s, stopWords=False):
    s = s.lower().strip()
    #s = re.sub(r'[^a-z0-9\s]',' ',s)
    if stopWords:
        s = remove_stopwords(s)
    return s

def get_cleaned(df, stopWords=False):
    sents = df[["Title"]]
    cleaned_sentences = []
    for index, row in df.iterrows():
        cleaned = clean_sentence(row["Title"], stopWords)
        cleaned_sentences.append(cleaned)
    return cleaned_sentences
cleaned_s_with_not_stopword = get_cleaned(df, stopWords=True)

In [5]:
from bert_serving.client import BertClient
bc = BertClient()
#run this cell only once !!!!!!!!!!!!!!!!!!!!!!
questions_embedding = []
for question_ in cleaned_s_with_not_stopword:
    questions_embedding.append(bc.encode([question_]))

C:\Users\Maha\anaconda3\envs\bert\lib\site-packages\bert_serving\client\__init__.py:299: UserWarning: some of your sentences have more tokens than "max_seq_len=25" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


In [24]:
def retrieveAndPrintAnswer(question_embedding, sentence_embeddings, df, sentences):
    max_sim = -1
    index_sim = -1
    potential_questions_indexes = []
    potential_questions = []
    answers = {}
    for index, faq_embedding in enumerate(sentence_embeddings):
        sim = cosine_similarity(faq_embedding, question_embedding)[0][0]    
        if sim>max_sim:
            max_sim = sim
            index_sim = index
            potential_questions_indexes.append(index_sim)
            potential_questions.append(df.iloc[index_sim,0])
    if(len(potential_questions)>11):
        potential_questions_indexes = potential_questions_indexes[1:11]
        potential_questions= potential_questions[1:11]
    potential_questions_indexes.reverse()
    potential_questions.reverse()
    answers = dict( zip(potential_questions_indexes,potential_questions))
    return answers
#     print("\n")
#     print("Question:", question)
#     print("\n")
#     print("Question retrieved:", df.iloc[index_sim,1])
#     print("\n")
#     print("Answer retrieved:",df.iloc[index_sim,2])

In [25]:
def getAnswers(question):
    question = clean_sentence(question, stopWords=True)
    question_embedding = bc.encode([question])
    answers = retrieveAndPrintAnswer(question_embedding, questions_embedding, df, cleaned_s_with_not_stopword)
    return answers

In [26]:
answers = getAnswers("Add Variable Editor to Custom Table")

In [27]:
import json
print(json.dumps(answers, sort_keys=False, indent=4))

{
    "286": "Add Variable Editor to Custom Table",
    "12": "Display Variable Editor in Service Portal",
    "9": "Update of multiple records using glide record script",
    "3": "Create object from XML attachment larger than 5MB",
    "0": "how to cut / trim the value short_description records"
}
